In [3]:
import pandas as pd
import numpy as np

In [1]:
# comment out unless google colab
from google.colab import files
uploaded = files.upload()

Saving final_25k.csv to final_25k.csv


In [5]:
# comment out unless google colab
import io
df = pd.read_csv(io.BytesIO(uploaded['final_25k.csv']))

In [38]:
df['url'] = df['file_name'].apply(lambda x: 'https://commons.wikimedia.org/wiki/'+x.replace(" ","_"))

In [40]:
df1 = df[['url', 'depicts', 'parsed caption']]

In [42]:
df1['suggestions'] = None

<ipython-input-42-d99b59a87583>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['suggestions'] = None


In [18]:
# https://towardsdatascience.com/keyword-extraction-process-in-python-with-natural-language-processing-nlp-d769a9069d5c
# https://liaad.github.io/yake/docs/getting_started.html

In [19]:
!pip install yake

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.2/60.2 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 20.9 MB/s eta 0:00:00


In [20]:
import yake

In [45]:
for i in range(len(df1)):
  kw_extractor = yake.KeywordExtractor()
  text = str(df1["parsed caption"][i])
  language = "en"
  max_ngram_size = 3
  deduplication_threshold = 0.9
  numOfKeywords = 8
  custom_kw_extractor = yake.KeywordExtractor(lan=language, n=max_ngram_size, dedupLim=deduplication_threshold, top=numOfKeywords, features=None)
  keywords = custom_kw_extractor.extract_keywords(text)

  suggestions=[]
  for kw in keywords:
    suggestions.append(kw)
  df1['suggestions'][i] = suggestions

Streaming output truncated to the last 5000 lines.
<ipython-input-45-5368f0d2ce97>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['suggestions'][i] = suggestions
<ipython-input-45-5368f0d2ce97>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['suggestions'][i] = suggestions
<ipython-input-45-5368f0d2ce97>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['suggestions'][i] = suggestions
<ipython-input-45-5368f0d2ce97>:14

In [51]:
def bests(suggestions):
  return [(term, score) for term, score in suggestions if score < 0.05]

In [52]:
df1['best_suggest'] = df1['suggestions'].apply(bests)

<ipython-input-52-1c296765629f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['best_suggest'] = df1['suggestions'].apply(bests)


In [72]:
import ast

df1['depicts'] = df1['depicts'].apply(ast.literal_eval)

<ipython-input-72-fd6255286db4>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['depicts'] = df1['depicts'].apply(ast.literal_eval)


In [107]:
def calc_percent(depicts, suggestions):
  if not depicts or not suggestions:
    return 0
  unique_depicts = set(depict.lower() for depict in depicts)
  matching = [item[0] for item in suggestions if item[0].lower() in unique_depicts]
  return (len(matching))/ len(depicts)

In [108]:
def calc_percent_extra_suggest(depicts, suggestions):
  if not depicts or not suggestions:
    return 0
  unique_depicts = set(depicts.lower() for depict in depicts)
  matching = [item[0] for item in suggestions if item[0].lower() in unique_depicts]
  extra_suggests = len(suggestions)-len(matching)
  return (len(matching) + (0.5*extra_suggests))/ len(depicts)

In [109]:
df1['depicts_suggested_percent'] = df1.apply(lambda x: calc_percent(x['depicts'], x['best_suggest']), axis=1)
df1['extra_suggest_score'] = df1.apply(lambda x: calc_percent_extra_suggest(x['depicts'], x['best_suggest']), axis=1)

<ipython-input-109-134bec963801>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['depicts_suggested_percent'] = df1.apply(lambda x: calc_percent(x['depicts'], x['best_suggest']), axis=1)
<ipython-input-109-134bec963801>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['extra_suggest_score'] = df1.apply(lambda x: calc_percent_extra_suggest(x['depicts'], x['best_suggest']), axis=1)


In [110]:
df1['depicts_suggested_percent'].value_counts()

depicts_suggested_percent
0.000000    20955
1.000000     2690
0.500000      566
0.333333      316
0.250000      238
0.200000       99
0.166667       44
0.666667       26
0.142857       14
0.125000        9
0.111111        6
0.100000        5
0.400000        4
0.285714        4
0.222222        3
0.800000        3
0.090909        3
0.750000        2
0.083333        2
0.066667        2
0.076923        2
0.038462        1
0.888889        1
0.071429        1
0.444444        1
0.055556        1
0.300000        1
0.030303        1
Name: count, dtype: int64

In [111]:
df1['extra_suggest_score'].value_counts()

extra_suggest_score
0.500000    2876
1.500000    2191
1.000000    2125
2.000000    1928
0.000000    1916
            ... 
0.264706       1
0.566667       1
0.074074       1
0.272727       1
0.033333       1
Name: count, Length: 162, dtype: int64

In [113]:
df1['suggest_count'] = df1['suggestions'].apply(lambda x: len(x))
df1['best_count'] = df1['best_suggest'].apply(lambda x: len(x))

<ipython-input-113-92a0baf7a5e4>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['suggest_count'] = df1['suggestions'].apply(lambda x: len(x))


In [115]:
df1['suggest_count'].value_counts()

suggest_count
20    9914
13    1311
3     1202
0     1157
16    1082
14    1039
15     848
6      812
12     768
7      736
17     729
11     679
19     668
10     653
8      638
9      629
5      583
18     565
4      421
1      323
2      243
Name: count, dtype: int64

In [116]:
df1['best_count'].value_counts()

best_count
3     3719
2     3430
1     3090
4     2589
0     1916
5     1565
7     1302
9     1296
6     1183
8     1028
10     895
11     654
12     498
13     396
20     360
14     292
15     245
16     193
18     129
17     124
19      96
Name: count, dtype: int64